<img src="https://www.strath.ac.uk/media/1newwebsite/webteam/logos/xUoS_Logo_Horizontal.png.pagespeed.ic.M6gv_BmDx1.png" width="350" align="left">

# Hardware Accelerated Spectrum Analysis on RFSoC
----

<div class="alert alert-box alert-info">
Please use Jupyter labs http://board_ip_address/lab for this notebook.
</div>

This notebook presents a flexible hardware accelerated Spectrum Analyser Module for the Zynq UltraScale+ RFSoC (rfsoc-sam). The steps outlined in the Table of Contents below will showcase the features and flexibility of the Spectrum Analyser and detail the underlying FPGA architecture.

The Spectrum Analyser Module was developed by the [University of Strathclyde](https://github.com/strath-sdr). The analyser is capable of inspecting 1638.4MHz of bandwidth and can achieve a maximum spectral resolution of 0.48828125kHz. The bandwidth is adjustable between 1638.4MHz and 3.2MHz, and the range of inspection is between 0 to 4096MHz using higher order Nyquist techniques. The spectrum analyser architecture supports several windowing coefficients through reprogrammable hardware. FPGA floating-point arithmetic is also exploited to accurately implement hardware accelerated spectrum calculations, including dB conversion for analysis. Plotting and animating the Spectrum is achieved using the [Plotly](https://plot.ly) libraries. The Plotly library allows the spectrum to be visualised in two different ways; a power spectrum frame plot and a spectrogram plot (also known as a waterfall).

The FPGA spectrum analyser design was developed using the model based design tool *HDL Coder* from The Mathworks. The FPGA decimation design was developed using Xilinx *System Generator*. All other FPGA design infrastructure was developed using the Xilinx Vivado Design Suite. The software design was enabled through the Python Productivity for Zynq (PYNQ) framework.


## Aims
* Review the design of the hardware accelerated Spectrum Analyser Module.
* Investigate the flexible decimator, reprogrammable window, and receiver.
* Perform hardware accelerated Spectrum Analysis with visualisation.

## Table of Contents
1. [Introduction](#introduction)<br>
    1.1. [Hardware Setup](#hardware-setup)<br>
    1.2. [Software Setup](#software-setup)<br>
2. [Simple Tone Generation](#simple-tone-generation)<br>
3. [Flexible Bandwidth Control](#flexible-bandwidth-control)<br>
4. [The Reprogrammable Window](#the-reprogrammable-window)<br>
5. [The RF ADC Receiver](#the-rf-adc-receiver)<br>
6. [Spectrum Analysis](#spectrum-analysis)<br>
    6.1. [The Spectrum Viewer](#the-spectrum-viewer)<br>
    6.2. [The Spectrogram (Waterfall)](#the-spectrogram)<br>
7. [Summary](#summary)
    
## References
* [Xilinx, Inc, "USP RF Data Converter: LogiCORE IP Product Guide", PG269, v2.3, June 2020](https://www.xilinx.com/support/documentation/ip_documentation/usp_rf_data_converter/v2_3/pg269-rf-data-converter.pdf)

## Revision History
* **v1.0** | *David Northcote, Lewis McLaughlin* | 11/02/2021 | Spectrum analyser notebook

----

## 1. Introduction <a class="anchor" id="introduction"></a>
The Zynq RFSoC contains high frequency samplers known as RF Data Converters (RF DCs). The RF DCs are tightly coupled with the Programmable Logic (PL), creating a high-throughput, low-latency path between the FPGA and analogue world. The Spectrum Analyser Module employs the RF Analogue-to-Digital Converters (RF ADCs) to receive RF time domain signals. The received data is manipulated using spectral pre-processing techniques in the PL, to prepare it for frequency domain analysis.

To efficiently process the RF spectrum, the Spectrum Analyser Module uses several features of the RF ADC blocks. This includes the flexible Digital Down Converter (DDC), which allows the analyser to perform dynamic centre frequency selection, and bandwidth control up to a decimation factor of 8x. The signal bandwidth is further controlled by a flexible decimator in the FPGA logic fabric, capable of interfacing to Super Sample Rate (SSR) signals.

The Spectrum Analyser Module contains a hardware accelerated Fast Fourier Transform (FFT) core, which can convert the RF sampled signal to the frequency domain using a range of different FFT lengths, $N = 64$ upto $N = 8192$. The frequency domain signal is further manipulated using a custom floating point processor to obtain the representative Power Spectral Density (PSD) or Power Spectrum. Furthermore, a hardware accelerated decibel (dB) converter is also used to condition the frequency domain signal for visual analysis.

A significant portion of the design has been implemented in the RFSoC's PL to prevent the Processing System (PS) from applying highly computational arithemtic. Furthermore, as the RF data is streaming from the RF ADC block to the PL, the FPGA logic fabric is better suited to manipulating the data in real-time. Spatial parallelism can also be exploited, allowing a Spectrum Analyser Module to be implemented for each available channel in the design without affecting the processing resources of the PS.

[Figure 1](#fig-1) presents a simple diagram illustrating the system overview. There is a Spectrum Analyser Module for each available RF ADC channel in the design. The Spectrum Analysers are also interfaced to their very own flexible decimator, allowing different sample rates to be configured for each channel. To further improve performance, the Spectrum Analysers automatically write frames of frequency domain data into external memory when they are ready. This prevents the processor from sending data requests to the PL, which is a common setup used by AXI Direct Memory Access (DMA) cores.

<a class="anchor" id="fig-1"></a>
<figure>
<img src='' height='50%' width='50%'/>
    <figcaption><b>Figure 1: The RFSoC Spectrum Analyser system overview.</b></figcaption>
</figure>

### 1.1. Hardware Setup <a class="anchor" id="hardware-setup"></a>
Your RFSoC2x2 development board can host two Spectrum Analyser Modules. There are four SMA interfaces on your board that are labelled DAC1, DAC2, ADC1, and ADC2. To setup your board for this demonstration, you can connect each channel in loopback as shown to the left of [Figure 2](#fig-2), or connect an antenna to one of the channels as shown to the right of Figure 2.

Don't worry if you don't have an antenna. The default loopback configuration will still be very interesting and is connected as follows:
* Channel 1: DAC1 to ADC1
* Channel 2: DAC2 to ADC2

<a class="anchor" id="fig-2"></a>
<figure>
<img src='' height='50%' width='50%'/>
    <figcaption><b>Figure 2: RFSoC2x2 development board setup, (left) loopback mode, (right) with an antenna.</b></figcaption>
</figure>

If you have chosen to use an antenna, you should connect channel 1 in loopback mode and connect the antenna to ADC2. This will be useful for the demonstrations later in this notebook. **Do Not** attach your antenna to any SMA interfaces labelled DAC1 or DAC2.

<div class="alert alert-box alert-danger">
<b>Caution:</b>
    In this demonstration, we generate tones using the RFSoC development board. Your device should be setup in loopback mode. You should understand that the RFSoC platform can also transmit RF signals wirelessly. Remember that unlicensed wireless transmission of RF signals may be illegal in your geographical location. Radio signals may also interfere with nearby devices, such as pacemakers and emergency radio equipment. Note that it is also illegal to intercept and decode particular RF signals. If you are unsure, please seek professional support.
</div>

### 1.2. Software Setup <a class="anchor" id="software-setup"></a>
We're nearly finished setting up the demonstration system. The majority of the libraries used by the Spectrum analyser design is contained inside the RFSoC-SAM software package. We only need to run a few code cells to initialise the software environment.

The primary module for loading the Spectrum Analyser design is contained inside `rfsoc_sam.overlay`. The class we are interested in using is `Overlay()`. During initialisation the class downloads the Spectrum Analyser bitstream to the PL and configures the RF DCs and FPGA IP cores contained in our system. This process may take around a minute to complete.

**Run** the code cell below to load the RFSoC-SAM Overlay class.

In [ ]:
from rfsoc_sam.overlay import Overlay

sam = Overlay()

When the RFSoC-SAM Overlay class is initialising, the setup script will also program the LMK and LMX low-jitter clock chips on the RFSoC2x2 to 122.8MHz and 409.6MHz respectively. The LMK clock is used as an input clock to the LMX clock. The LMX clock is then used as an input clock to the RF ADC tile's internal phased-locked loop (PLL). The RF ADC tile's PLL generates several other clocks required by the RF ADC blocks in the tile, which allow them to achieve RF sampling.

----

## 2. Simple Tone Generation <a class="anchor" id="simple-tone-generation"></a>

As previously illustrated in [Figure 1](#fig-1), a simple amplitude controller is required to generate tones with the RF Digital-to-Analogue Converters (RF DACs). We use tone generation in this demonstration as a means to provide a signal for the user to inspect when using the Spectrum Analyser Module. 

As demonstrated in [Figure 3](#fig-3), the amplitude controller applies a value to the input of the RF DAC. The Numerically Controlled Oscillators (NCOs) in the RF DAC's fine mixer generates a tone and the signal is transferred to the outside world through the digital-to-analogue sampler. In the current setup, the signal should be looped back around into the RF ADC for spectral analysis.

<a class="anchor" id="fig-3"></a>
<figure>
<img src='' height='50%' width='50%'/>
    <figcaption><b>Figure 3: Tone generation using the amplitude controller and RF DAC's fine mixer.</b></figcaption>
</figure>

To simulate this process, a basic tone generator application has been created in software. Run the code cell below to start the tone generator application. A receiver time domain plot and a frequency domain plot will appear. Each plot presents the generated tone after Digital Down Conversion (DDC).

In [1]:
from rfsoc_sam.software_apps import ToneGeneratorApplication

tone_application = ToneGeneratorApplication()
tone_application.display()

    'data': [{'name': 'Real',
              'type': 'scatter',
              'ui…

Use the slider above to change the transmitter frequency. The tone generator application has been set-up to operate using a sample frequency of 2048MHz and an FFT size of 2048. The receiver's centre frequency is set to 1024MHz and a small noise generator has been used to apply noise to the received time domain signal. When changing the transmitter frequency using the slider, you should see that the time domain signal oscillates slower when approaching a transmitter frequency of 1024MHz, and oscillates faster when moving away from 1024MHz.

The example above demonstrates the fundamental principles of operation for the hardware accelerated Spectrum Analyser Module. The RF ADC receiver should be tuned to an appropriate centre frequency, and since the signal is complex, both sides of the spectrum can be used for spectral analysis.

## 3. Flexible Bandwidth Control <a class="anchor" id="flexible-bandwidth-control"></a>
Why is bandwidth control important for spectral analysis? If the sample frequency is fixed at all times, the only way to improve spectral resolution would be to increase the size of the FFT used by the spectrum analyser. Changing the FFT size is fairly trivial using a software approach, however, when using a hardware accelerated FFT, there are limitations on FPGA resource allocation and system control. An alternative approach is to dynamically change the sample frequency of the input time domain signal, and since the signal bandwidth is directly related to sample frequency, the task becomes a little less complex.

The sample frequency of a time domain signal can be changed using Finite Impulse Response (FIR) decimation filters. Changing the sample frequency through decimation techniques is far more effective than increasing the size of the FFT used in the hardware design. There are trade-offs between memory bandwidth, and FPGA resource allocation to consider, and these favour using FIR filters rather than large FFT architectures. Therefore, the Spectrum Analyser Module in this demonstration only uses FIRs to change the sample frequency and improve spectral resolution.

The flexible bandwidth controller is a fairly straight forward hardware architecture. In [Figure 1](#fig-1) you can see that it is directly interfaced to the RF ADC block. Before any decimation is performed in the RFSoC's PL, the RF ADC decimators are first applied to decrease the SSR size. The signal that is interfaced to the RFSoC's PL is a sample frequency of 1024MHz, using an SSR 2. A cascade of efficient halfband decimation filters are then implemented to decrease the rate of the signal. An illustration of the bandwidth controller is presented in [Figure 4](#fig-4).

<a class="anchor" id="fig-4"></a>
<figure>
<img src='' height='50%' width='50%'/>
    <figcaption><b>Figure 4: The architecture of the flexibile bandwidth controller.</b></figcaption>
</figure>

We should explore the concept of bandwidth selection a little more by using another software simulation. Lets set-up a cascade of FIR decimators and investigate the impact of the input signal bandwidth and spectral resolution as it is decimated. For this simulation, the output signal is best explored in the frequency domain. Run the bandwidth controller application by executing the code cell below.

## 4. The Reprogrammable Window <a class="anchor" id="the-reprogrammable-window"></a>

## 5. The RF ADC Receiver <a class="anchor" id="the-rf-adc-receiver"></a>

## 6. Spectrum Analysis <a class="anchor" id="spectrum-analysis"></a>

### 6.1. The Spectrum Viewer <a class="anchor" id="the-spectrum-viewer"></a>

### 6.2. The Spectrogram (Waterfall) <a class="anchor" id="the-spectrogram"></a>

## 7. Summary <a class="anchor" id="summary"></a>